# BERT Reranker Inference on AWS Inferentia2 with Triton Inference Server

This notebook deploys the [Alibaba-NLP/gte-multilingual-reranker-base](https://huggingface.co/Alibaba-NLP/gte-multilingual-reranker-base)
BERT cross-encoder reranker (~306M parameters) on an AWS **inf2.8xlarge** instance using
NVIDIA Triton Inference Server with the AWS Neuron SDK, then benchmarks throughput and latency.

Everything is self-contained: the notebook writes all required files (Dockerfile, Triton config,
Python backend, compile script, benchmark client), builds the Docker image, compiles the models,
starts the server, and runs the benchmark.

**Instance**: inf2.8xlarge (1 Neuron device, 2 NeuronCores, 32 GB device memory)  
**Time to complete**: ~60-80 minutes (compilation dominates on first run)  
**Prerequisites**: Deep Learning AMI Neuron (Ubuntu 24.04), Docker installed

## Methodology

We measure throughput (inferences/second) and latency (P50, P95, P99) under multiple load patterns,
varying both client-side batch size and request concurrency to exercise Triton's dynamic batching.

- **Input**: Query-passage pairs, tokenized and padded to 1024 tokens (`input_ids` + `attention_mask`)
- **Output**: Single float reranking score per sample (logit from cross-encoder head)
- **Duration**: 10 seconds per test configuration
- **Warmup**: 5 requests before timing
- **Concurrency**: Python threads, one HTTP client per thread

### Neuron-Specific: Per-Batch-Size Compilation

Unlike GPUs, Neuron models are compiled for a **fixed batch size**. Running a BS=1 request through
a BS=16 compiled model wastes 15/16 of the compute. We compile separate models for BS=1, 2, 4, 8, 16
and the Triton Python backend dispatches each request to the smallest model that fits.

### Triton Configuration

- **2 model instances** (one per NeuronCore), each pinned via `NEURON_RT_VISIBLE_CORES`
- **Dynamic batching**: preferred sizes [4, 8, 16], max queue delay 5ms
- **Python backend**: Triton has no native Neuron backend, so we use the Python backend with `torch_neuronx`

### Test Matrix

| Test | Concurrency | Client Batch Sizes |
|------|-------------|--------------------|
| Baseline | 1 (no batching) | 1 |
| Dynamic batching | 4, 8, 16 | 1, 2, 4, 8, 16 |

## Model & Instance Specifications

| Spec | Value |
|------|-------|
| Model | Alibaba-NLP/gte-multilingual-reranker-base (~306M params) |
| Architecture | 12-layer NewBERT cross-encoder, hidden size 768 |
| Task | Cross-encoder reranking (query + passage -> relevance score) |
| Input | `input_ids`, `attention_mask` (1024 tokens, padded) |
| Output | Single float logit score per sample |
| Compiled model size | ~720 MB per batch size variant |
| Instance type | inf2.8xlarge |
| Neuron device | 1 device, 2 NeuronCores |
| Device memory | 32 GB (16 GB per core) |
| vCPUs / RAM | 32 / 128 GB |

### Software Stack

| Component | Version |
|-----------|--------|
| Neuron SDK | 2.27.1 |
| PyTorch / torch-neuronx | 2.9.0 / 2.9.0.2.11 |
| neuronx-cc (compiler) | 2.22.12471 |
| Transformers | 4.48.0 |
| Triton Inference Server | 2.65.0 (r26.01, built from source) |
| Python | 3.12 |
| OS / AMI | Ubuntu 24.04 / Deep Learning AMI Neuron 20260126 |

> **Important**: Transformers versions 4.54.0+ have a confirmed ~20% performance regression for
> this model family on Neuron. Use versions 4.48.0 through 4.53.3.

---
## Step 0: Setup & Environment Check

After you deploy the instance using the Ubuntu Neuron Deep Learning AMI (with all the Neuron drivers installed), run this notebook inside the pre-installed PyTorch 2.9 Neuron virtual environment:

```bash
source /opt/aws_neuronx_venv_pytorch_2_9/bin/activate
pip install jupyter
jupyter notebook --ip=0.0.0.0 --no-browser
```

Alternatively, if you are running from within a remote VSCode instance, you can use `ln -s /opt/aws_neuronx_venv_pytorch_2_9/bin/activate ~/.venv` to help VSCode find your kernel.

In [1]:
import subprocess, sys, os, time, shutil

# Verify Neuron environment
import torch, torch_neuronx
print(f'PyTorch: {torch.__version__}')
print(f'torch-neuronx: {torch_neuronx.__version__}')

# Ensure correct transformers version
try:
    import transformers
    ver = transformers.__version__
    print(f'transformers: {ver}')
    if ver >= '4.54.0':
        print('WARNING: transformers >= 4.54.0 has ~20% regression for this model. Downgrading...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install',
                               'transformers==4.48.0', '-q'])
except ImportError:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install',
                           'transformers==4.48.0', '-q'])

# Install Triton client
subprocess.check_call([sys.executable, '-m', 'pip', 'install',
                       'tritonclient[http]', '-q'])

# Show Neuron devices
r = subprocess.run(['neuron-ls'], capture_output=True, text=True)
print(f'\n{r.stdout}')

PyTorch: 2.9.0+cu128
torch-neuronx: 2.9.0.2.11.19912+e48cd891


transformers: 4.53.3



instance-type: inf2.8xlarge
instance-id: i-06e983546fee50417
+--------+--------+----------+--------+--------------+----------+------+
| NEURON | NEURON |  NEURON  | NEURON |     PCI      |   CPU    | NUMA |
| DEVICE | CORES  | CORE IDS | MEMORY |     BDF      | AFFINITY | NODE |
+--------+--------+----------+--------+--------------+----------+------+
| 0      | 2      | 0-1      | 32 GB  | 0000:00:1f.0 | 0-31     | -1   |
+--------+--------+----------+--------+--------------+----------+------+



---
## Step 1: Compile BERT Reranker for All Batch Sizes

Compile 5 model variants (BS=1, 2, 4, 8, 16) with sequence length 1024.
Takes ~10 minutes per variant (~50 min total on inf2). Skips already-compiled models.

The `--auto-cast matmult` flag casts matrix multiplications to BF16, yielding 57-120% speedup
with negligible accuracy impact (<0.0004).

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

os.environ['NEURON_RT_LOG_LEVEL'] = 'ERROR'

MODEL_ID = 'Alibaba-NLP/gte-multilingual-reranker-base'
SEQ_LENGTH = 1024
MODEL_DIR = os.path.expanduser('~/triton_repo/bert_reranker/1')
BATCH_SIZES = [1, 2, 4, 8, 16]

os.makedirs(MODEL_DIR, exist_ok=True)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID, torchscript=True, trust_remote_code=True)
model.eval()

print(f'Model: {MODEL_ID}')
print(f'Parameters: {sum(p.numel() for p in model.parameters()):,}')
print(f'Compiling for batch sizes: {BATCH_SIZES}')
print(f'Sequence length: {SEQ_LENGTH}\n')

for bs in BATCH_SIZES:
    output_path = os.path.join(MODEL_DIR, f'model_bs{bs}.pt')
    if os.path.exists(output_path):
        print(f'  BS={bs}: already compiled, skipping')
        continue
    print(f'  Compiling BS={bs}...')
    queries = ['What is machine learning?'] * bs
    docs = ['Machine learning is a subset of artificial intelligence.'] * bs
    inputs = tokenizer(queries, docs, return_tensors='pt',
                       max_length=SEQ_LENGTH, padding='max_length', truncation=True)
    t0 = time.time()
    model_neuron = torch_neuronx.trace(
        model,
        (inputs['input_ids'], inputs['attention_mask']),
        compiler_args=['--model-type', 'transformer', '--optlevel', '2',
                       '--auto-cast', 'matmult'])
    compile_time = time.time() - t0
    torch.jit.save(model_neuron, output_path)
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f'    Saved: {output_path} ({size_mb:.1f} MB, {compile_time:.0f}s)')

print('\nCompiled models:')
for f in sorted(os.listdir(MODEL_DIR)):
    if f.endswith('.pt'):
        size = os.path.getsize(os.path.join(MODEL_DIR, f)) / (1024 * 1024)
        print(f'  {f}: {size:.1f} MB')

Model: Alibaba-NLP/gte-multilingual-reranker-base
Parameters: 305,959,681
Compiling for batch sizes: [1, 2, 4, 8, 16]
Sequence length: 1024

  Compiling BS=1...


.

.

.

.

...

.

.

Completed run_backend_driver.



Compiler status PASS


    Saved: /home/ubuntu/triton_repo/bert_reranker/1/model_bs16.pt (715.2 MB, 472s)

Compiled models:
  model_bs1.pt: 676.4 MB
  model_bs16.pt: 715.2 MB
  model_bs2.pt: 679.1 MB
  model_bs4.pt: 684.3 MB
  model_bs8.pt: 695.8 MB


---
## Step 2: Write Triton Model Repository Files

Write `config.pbtxt` and `model.py` into the model repository alongside the compiled `.pt` files.

Key configuration for inf2.8xlarge:
- **2 model instances** (one per NeuronCore)
- **No LNC configuration** needed (LNC is a trn2-only feature)
- **Output**: single float score (`score`)
- **Sequence length**: 1024

In [3]:
REPO_DIR = os.path.expanduser('~/triton_repo/bert_reranker')

# -- config.pbtxt (inf2.8xlarge: 2 instances) -----------------------------------
config_pbtxt = r"""name: "bert_reranker"
platform: "python"
backend: "python"
max_batch_size: 16

input [
  {
    name: "input_ids"
    data_type: TYPE_INT64
    dims: [1024]
  },
  {
    name: "attention_mask"
    data_type: TYPE_INT64
    dims: [1024]
  }
]

output [
  {
    name: "score"
    data_type: TYPE_FP32
    dims: [1]
  }
]

instance_group [
  {
    count: 2
    kind: KIND_CPU
  }
]

dynamic_batching {
  preferred_batch_size: [4, 8, 16]
  max_queue_delay_microseconds: 5000
}

parameters: {
  key: "model_dir"
  value: { string_value: "/models/bert_reranker/1" }
}

parameters: {
  key: "tokenizer_name"
  value: { string_value: "Alibaba-NLP/gte-multilingual-reranker-base" }
}

parameters: {
  key: "max_seq_length"
  value: { string_value: "1024" }
}
"""

with open(os.path.join(REPO_DIR, 'config.pbtxt'), 'w') as f:
    f.write(config_pbtxt)
print('Wrote config.pbtxt (instance_group count=2 for inf2.8xlarge)')

# -- model.py (Triton Python backend, inf2) ------------------------------------
model_py = r'''#!/usr/bin/env python3
"""
Triton Python Backend for BERT Reranker on AWS Neuron (inf2)

Loads compiled models for each batch size (1, 2, 4, 8, 16) and
dispatches to the best-fit model to avoid padding waste.
Each Triton instance is pinned to a separate NeuronCore (cores 0-1).
"""

import os
import json
import numpy as np

try:
    import triton_python_backend_utils as pb_utils
except ImportError:
    pass

import torch
import torch_neuronx


class TritonPythonModel:
    BATCH_SIZES = [1, 2, 4, 8, 16]

    def initialize(self, args):
        self.model_config = json.loads(args["model_config"])
        params = self.model_config.get("parameters", {})
        model_dir = params.get("model_dir", {}).get(
            "string_value", "/models/bert_reranker/1")
        self.max_seq_length = int(
            params.get("max_seq_length", {}).get("string_value", "1024"))

        # Pin this instance to a specific NeuronCore (0-1 for inf2.8xlarge)
        instance_name = args.get("model_instance_name", "bert_reranker_0_0")
        core_id = int(instance_name.split("_")[-1])
        os.environ["NEURON_RT_VISIBLE_CORES"] = str(core_id)
        print(f"Instance {instance_name}: pinned to NeuronCore {core_id}")

        # Load all compiled models
        self.models = {}
        for bs in self.BATCH_SIZES:
            model_path = os.path.join(model_dir, f"model_bs{bs}.pt")
            if os.path.exists(model_path):
                self.models[bs] = torch.jit.load(model_path)
                self.models[bs].eval()
                print(f"  Loaded model for batch_size={bs}")
            else:
                print(f"  WARNING: Model not found: {model_path}")

        if not self.models:
            raise RuntimeError("No compiled models found!")

        # Warmup all models
        print("Warming up models...")
        for bs, mdl in self.models.items():
            dummy_ids = torch.zeros((bs, self.max_seq_length), dtype=torch.long)
            dummy_mask = torch.ones((bs, self.max_seq_length), dtype=torch.long)
            with torch.no_grad():
                for _ in range(3):
                    _ = mdl(dummy_ids, dummy_mask)
            print(f"  Warmed up BS={bs}")
        print(f"Model initialization complete! "
              f"Available batch sizes: {sorted(self.models.keys())}")

    def _get_best_model(self, actual_batch_size):
        for bs in self.BATCH_SIZES:
            if bs >= actual_batch_size and bs in self.models:
                return bs, self.models[bs]
        largest = max(self.models.keys())
        return largest, self.models[largest]

    def execute(self, requests):
        responses = []
        for request in requests:
            input_ids = torch.from_numpy(
                pb_utils.get_input_tensor_by_name(request, "input_ids").as_numpy()
            ).long()
            attention_mask = torch.from_numpy(
                pb_utils.get_input_tensor_by_name(request, "attention_mask").as_numpy()
            ).long()

            actual_bs = input_ids.shape[0]
            target_bs, model = self._get_best_model(actual_bs)

            # Pad input tensors to compiled batch size if needed
            if actual_bs < target_bs:
                pad = target_bs - actual_bs
                input_ids = torch.cat([input_ids,
                    torch.zeros((pad, input_ids.shape[1]), dtype=torch.long)], dim=0)
                attention_mask = torch.cat([attention_mask,
                    torch.zeros((pad, attention_mask.shape[1]), dtype=torch.long)], dim=0)

            with torch.no_grad():
                outputs = model(input_ids, attention_mask)

            # Extract logit score: outputs is a tuple, first element is logits
            if isinstance(outputs, tuple):
                logits = outputs[0]
            else:
                logits = outputs
            scores = logits[:actual_bs, 0:1].cpu().numpy()

            output_tensor = pb_utils.Tensor("score",
                                            scores.astype(np.float32))
            responses.append(
                pb_utils.InferenceResponse(output_tensors=[output_tensor]))
        return responses

    def finalize(self):
        print("Finalizing BERT Reranker model...")
        if hasattr(self, "models"):
            for bs, mdl in self.models.items():
                del mdl
            self.models.clear()
'''

with open(os.path.join(MODEL_DIR, 'model.py'), 'w') as f:
    f.write(model_py)
print('Wrote model.py (cores 0-1, 2 input tensors)')

# Verify
print('\nModel repository:')
for root, dirs, files in os.walk(os.path.expanduser('~/triton_repo')):
    level = root.replace(os.path.expanduser('~/triton_repo'), '').count(os.sep)
    indent = '  ' * level
    print(f'{indent}{os.path.basename(root)}/')
    for f in sorted(files):
        size = os.path.getsize(os.path.join(root, f)) / (1024 * 1024)
        label = f'{f} ({size:.1f} MB)' if size > 1 else f
        print(f'{indent}  {label}')

Wrote config.pbtxt (instance_group count=2 for inf2.8xlarge)
Wrote model.py (cores 0-1, 2 input tensors)

Model repository:
triton_repo/
  bert_reranker/
    config.pbtxt
    1/
      model.py
      model_bs1.pt (676.4 MB)
      model_bs16.pt (715.2 MB)
      model_bs2.pt (679.1 MB)
      model_bs4.pt (684.3 MB)
      model_bs8.pt (695.8 MB)


---
## Step 3: Build Triton + Neuron Docker Image

Triton has no native Neuron backend, so we build Triton from source inside the AWS Neuron
PyTorch inference base image. This takes **15-20 minutes** and produces a ~15.8 GB image.

The cell skips the build if the image already exists.

**Note**: We pin `transformers==4.48.0` inside the Docker image to avoid the performance
regression with versions >= 4.54.0.

In [4]:
DOCKER_IMAGE = 'triton-neuron-bert-reranker:latest'

# Check if already built
r = subprocess.run(['docker', 'images', '-q', DOCKER_IMAGE],
                   capture_output=True, text=True)
if r.stdout.strip():
    print(f'Docker image already exists: {r.stdout.strip()}')
    print('Delete with: docker rmi ' + DOCKER_IMAGE)
else:
    # Write Dockerfile
    dockerfile = r"""ARG BASE_IMAGE=public.ecr.aws/neuron/pytorch-inference-neuronx:2.9.0-neuronx-py312-sdk2.27.1-ubuntu24.04
FROM $BASE_IMAGE

ENV DEBIAN_FRONTEND=noninteractive \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    PJRT_DEVICE=NEURON \
    LD_LIBRARY_PATH="/opt/conda/lib:/opt/aws/neuron/lib:/lib/x86_64-linux-gnu:${LD_LIBRARY_PATH}" \
    PATH="/opt/program:/opt/aws/neuron/bin:/opt/tritonserver/bin:${PATH}"

RUN apt-get update && apt-get install -y --no-install-recommends \
    wget gnupg2 build-essential git nginx pkg-config unzip \
    libssl-dev libcurl4-openssl-dev libgoogle-perftools-dev \
    libnuma-dev libarchive-dev libxml2-dev zlib1g-dev \
    autoconf automake libtool gperf scons patchelf \
    libre2-dev libb64-dev rapidjson-dev libboost-dev \
    cmake cmake-data \
    && rm -rf /var/lib/apt/lists/*

RUN pip3 install --no-cache-dir --upgrade pip setuptools wheel virtualenv build cmake==3.31.10
RUN pip3 install transformers==4.48.0

RUN git clone --depth=1 --branch=r26.01 https://github.com/triton-inference-server/server.git /server && \
    cd /server && \
    Python3_ROOT_DIR=/opt/conda \
    Python3_EXECUTABLE=/opt/conda/bin/python3 \
    Python3_INCLUDE_DIR=/opt/conda/include/python3.12 \
    Python3_LIBRARY=/opt/conda/lib/libpython3.12.so \
    ./build.py -v --no-container-build --build-dir=/server/build --backend=python \
    --enable-metrics --enable-logging --enable-stats --endpoint="http" --endpoint="grpc" && \
    cp -r /server/build/opt/* /opt/ && \
    cd / && rm -rf /server

EXPOSE 8000 8001 8002
CMD ["tritonserver", "--model-repository=/models"]
"""
    dockerfile_path = os.path.expanduser('~/Dockerfile.triton-neuron')
    with open(dockerfile_path, 'w') as f:
        f.write(dockerfile)

    print('Building Triton + Neuron Docker image (15-20 minutes)...')
    r = subprocess.run(
        ['docker', 'build', '-f', dockerfile_path, '-t', DOCKER_IMAGE, '.'],
        cwd=os.path.expanduser('~'),
        capture_output=True, text=True, timeout=2400)
    if r.returncode == 0:
        print('Build complete!')
    else:
        print(f'Build FAILED (rc={r.returncode})')
        print(r.stderr[-3000:])

# Show image
r = subprocess.run(['docker', 'images', DOCKER_IMAGE], capture_output=True, text=True)
print(r.stdout)

Building Triton + Neuron Docker image (15-20 minutes)...


Build complete!
IMAGE                                ID             DISK USAGE   CONTENT SIZE   EXTRA
triton-neuron-bert-reranker:latest   3e169a9bfbe4       24.1GB         7.83GB        



---
## Step 4: Start Triton Server

Launch the Docker container with the Neuron device mounted and the model repository bind-mounted.
With 2 instances each loading 5 batch-size variants of a ~720 MB model, loading takes
~1-2 minutes.

In [5]:
import urllib.request

NUM_INSTANCES = 2  # inf2.8xlarge: 2 NeuronCores

# Stop any previous run
subprocess.run(['docker', 'rm', '-f', 'triton-bert-reranker'],
               capture_output=True)
time.sleep(3)

cmd = [
    'docker', 'run', '-d',
    '--name', 'triton-bert-reranker',
    '--device=/dev/neuron0',
    '-v', os.path.expanduser('~/triton_repo') + ':/models:ro',
    '-p', '8000:8000', '-p', '8001:8001', '-p', '8002:8002',
    DOCKER_IMAGE,
    'tritonserver', '--model-repository=/models', '--log-verbose=0',
]
r = subprocess.run(cmd, capture_output=True, text=True)
if r.returncode != 0:
    print(f'Failed to start container: {r.stderr}')
else:
    print(f'Container started. Waiting for {NUM_INSTANCES} model instances to load (~1-2 min)...')

# Poll for readiness (up to 6 minutes)
for i in range(180):
    time.sleep(2)
    try:
        resp = urllib.request.urlopen('http://localhost:8000/v2/health/ready', timeout=2)
        if resp.status == 200:
            elapsed = (i + 1) * 2
            print(f'Server ready after ~{elapsed}s')
            break
    except Exception:
        pass
else:
    print('Timeout waiting for server!')
    r = subprocess.run(['docker', 'logs', '--tail', '30', 'triton-bert-reranker'],
                       capture_output=True, text=True)
    print(r.stdout)

# Verify instances loaded
r = subprocess.run(['docker', 'logs', 'triton-bert-reranker'],
                   capture_output=True, text=True)
n = r.stdout.count('Model initialization complete')
print(f'Model instances initialized: {n}/{NUM_INSTANCES}')

Container started. Waiting for 2 model instances to load (~1-2 min)...


Server ready after ~86s
Model instances initialized: 2/2


---
## Step 5: Run Benchmark

Runs the full test matrix: 1 baseline + 15 dynamic-batching configurations (3 concurrency levels
x 5 batch sizes), 10 seconds each. Total runtime ~3 minutes.

Concurrency levels are 4/8/16, scaled for 2 NeuronCores (2x, 4x, 8x per core).

**Note**: The reranker uses seq_len=1024, so per-inference
latency is significantly higher than short-sequence models (~215ms vs ~0.37ms per sample).

**Note**: One worker per test will print a harmless greenlet thread-switch error. This is a
known cosmetic issue in `tritonclient` and does not affect results.

In [6]:
import numpy as np
from transformers import AutoTokenizer
import tritonclient.http as httpclient
import threading
from queue import Queue

TRITON_URL = 'localhost:8000'
MODEL_NAME = 'bert_reranker'
DURATION = 10.0  # seconds per test

tok = AutoTokenizer.from_pretrained('Alibaba-NLP/gte-multilingual-reranker-base',
                                    trust_remote_code=True)


def _make_inputs(batch_size):
    """Create tokenized inputs for the reranker (query + passage pairs)."""
    queries = ['What is machine learning?'] * batch_size
    docs = ['Machine learning is a subset of artificial intelligence that '
            'focuses on building systems that learn from data.'] * batch_size
    tokens = tok(queries, docs, max_length=1024, padding='max_length',
                 truncation=True, return_tensors='np')
    ids_np = tokens['input_ids'].astype(np.int64)
    mask_np = tokens['attention_mask'].astype(np.int64)
    return ids_np, mask_np


def _worker(client, batch_size, duration_s, latencies_q, stop_evt):
    ids_np, mask_np = _make_inputs(batch_size)
    inp_ids = httpclient.InferInput('input_ids', ids_np.shape, 'INT64')
    inp_mask = httpclient.InferInput('attention_mask', mask_np.shape, 'INT64')
    inp_ids.set_data_from_numpy(ids_np)
    inp_mask.set_data_from_numpy(mask_np)
    t_end = time.time() + duration_s
    while time.time() < t_end and not stop_evt.is_set():
        try:
            t0 = time.time()
            client.infer(model_name=MODEL_NAME,
                         inputs=[inp_ids, inp_mask])
            latencies_q.put((time.time() - t0) * 1000)
        except Exception:
            break


def run_concurrent(batch_size, num_workers, duration_s=DURATION):
    clients = [httpclient.InferenceServerClient(url=TRITON_URL)
               for _ in range(num_workers)]
    # Warmup
    ids_np, mask_np = _make_inputs(batch_size)
    inp_ids = httpclient.InferInput('input_ids', ids_np.shape, 'INT64')
    inp_mask = httpclient.InferInput('attention_mask', mask_np.shape, 'INT64')
    inp_ids.set_data_from_numpy(ids_np)
    inp_mask.set_data_from_numpy(mask_np)
    for _ in range(5):
        clients[0].infer(model_name=MODEL_NAME,
                         inputs=[inp_ids, inp_mask])

    q = Queue()
    stop = threading.Event()
    threads = []
    t_start = time.time()
    for i in range(num_workers):
        t = threading.Thread(target=_worker,
                             args=(clients[i], batch_size, duration_s, q, stop))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    total_time = time.time() - t_start

    latencies = []
    while not q.empty():
        latencies.append(q.get())
    if not latencies:
        return None
    return {
        'batch_size': batch_size, 'num_workers': num_workers,
        'total_requests': len(latencies),
        'p50': np.percentile(latencies, 50),
        'p95': np.percentile(latencies, 95),
        'p99': np.percentile(latencies, 99),
        'throughput': (len(latencies) * batch_size) / total_time,
    }


def run_baseline(duration_s=DURATION):
    client = httpclient.InferenceServerClient(url=TRITON_URL)
    ids_np, mask_np = _make_inputs(1)
    inp_ids = httpclient.InferInput('input_ids', ids_np.shape, 'INT64')
    inp_mask = httpclient.InferInput('attention_mask', mask_np.shape, 'INT64')
    inp_ids.set_data_from_numpy(ids_np)
    inp_mask.set_data_from_numpy(mask_np)
    for _ in range(10):
        client.infer(model_name=MODEL_NAME,
                     inputs=[inp_ids, inp_mask])
    latencies = []
    t_start = time.time()
    while time.time() - t_start < duration_s:
        t0 = time.time()
        client.infer(model_name=MODEL_NAME,
                     inputs=[inp_ids, inp_mask])
        latencies.append((time.time() - t0) * 1000)
    total_time = time.time() - t_start
    return {
        'batch_size': 1, 'num_workers': 1,
        'total_requests': len(latencies),
        'p50': np.percentile(latencies, 50),
        'p95': np.percentile(latencies, 95),
        'p99': np.percentile(latencies, 99),
        'throughput': len(latencies) / total_time,
    }


# -- Run all tests -------------------------------------------------------------
client = httpclient.InferenceServerClient(url=TRITON_URL)
assert client.is_server_ready(), 'Triton server not ready!'

print('BERT RERANKER TRITON BENCHMARK - Neuron (inf2.8xlarge)')
print('=' * 90)

results = []

# Baseline
print('\nBaseline: single request, no concurrency...')
r = run_baseline()
results.append(r)
print(f'  P50: {r["p50"]:.2f}ms  Throughput: {r["throughput"]:.1f} inf/sec')

# Dynamic batching -- concurrency levels scaled for 2 cores
for conc in [4, 8, 16]:
    print(f'\nConcurrency={conc}:')
    for bs in [1, 2, 4, 8, 16]:
        r = run_concurrent(bs, conc)
        if r:
            results.append(r)
            print(f'  BS={bs:<3} P50={r["p50"]:>8.2f}ms  '
                  f'P95={r["p95"]:>8.2f}ms  '
                  f'P99={r["p99"]:>8.2f}ms  '
                  f'Throughput={r["throughput"]:>8.1f} inf/sec')

# Summary table
print(f'\n{"=" * 90}')
print(f'{"Batch":<8}{"Workers":<10}{"Requests":<12}'
      f'{"P50 (ms)":<12}{"P95 (ms)":<12}{"P99 (ms)":<12}{"Throughput":<15}')
print('-' * 90)
for r in results:
    print(f'{r["batch_size"]:<8}{r["num_workers"]:<10}{r["total_requests"]:<12}'
          f'{r["p50"]:<12.2f}{r["p95"]:<12.2f}{r["p99"]:<12.2f}'
          f'{r["throughput"]:<15.1f}')

BERT RERANKER TRITON BENCHMARK - Neuron (inf2.8xlarge)

Baseline: single request, no concurrency...


  P50: 18.35ms  Throughput: 54.4 inf/sec

Concurrency=4:


  BS=1   P50=   43.18ms  P95=   43.85ms  P99=   44.21ms  Throughput=    69.3 inf/sec


  BS=2   P50=   50.52ms  P95=   51.73ms  P99=   52.08ms  Throughput=   143.0 inf/sec


  BS=4   P50=   59.18ms  P95=  104.65ms  P99=  104.95ms  Throughput=   146.5 inf/sec


...

  BS=1   P50=  100.88ms  P95=  138.16ms  P99=  140.64ms  Throughput=   156.6 inf/sec


  BS=2   P50=  151.76ms  P95=  273.53ms  P99=  278.05ms  Throughput=   160.1 inf/sec


  BS=4   P50=  389.66ms  P95=  434.89ms  P99=  438.58ms  Throughput=   146.4 inf/sec


  BS=8   P50=  990.37ms  P95= 1017.46ms  P99= 1048.21ms  Throughput=   126.1 inf/sec


  BS=16  P50= 1598.51ms  P95= 1786.68ms  P99= 1789.90ms  Throughput=   140.2 inf/sec

Batch   Workers   Requests    P50 (ms)    P95 (ms)    P99 (ms)    Throughput     
------------------------------------------------------------------------------------------
1       1         545         18.35       18.54       18.60       54.4           
1       4         696         43.18       43.85       44.21       69.3           
2       4         717         50.52       51.73       52.08       143.0          
4       4         370         59.18       104.65      104.95      146.5          
8       4         158         129.24      256.40      307.18      123.7          
16      4         90          445.37      451.51      452.35      138.5          
1       8         1450        51.20       53.61       56.10       144.5          
2       8         814         96.35       98.90       99.06       161.2          
4       8         379         162.54      270.46      271.27      147.8          
8  

---
## Step 6: Sample Reranking Results

Send a real query with multiple passages to the Triton server and display the reranking scores.
This demonstrates the model's ability to rank passages by relevance.

In [7]:
import tritonclient.http as httpclient
from transformers import AutoTokenizer

client = httpclient.InferenceServerClient(url=TRITON_URL)
tok = AutoTokenizer.from_pretrained('Alibaba-NLP/gte-multilingual-reranker-base',
                                    trust_remote_code=True)

query = 'What are the benefits of renewable energy?'
passages = [
    'Renewable energy sources like solar and wind power produce electricity without '
    'greenhouse gas emissions, reducing climate change impacts and improving air quality.',
    'The stock market experienced significant volatility in Q4 2025, with tech stocks '
    'leading the decline amid rising interest rates.',
    'Solar panels have become 90% cheaper over the past decade, making renewable '
    'energy cost-competitive with fossil fuels in most markets.',
    'The history of ancient Rome spans over a thousand years, from its founding '
    'in 753 BC to the fall of the Western Roman Empire in 476 AD.',
    'Wind energy creates more jobs per megawatt than coal or natural gas power plants, '
    'boosting local economies in rural areas.',
    'Python is a popular programming language known for its readability and '
    'extensive library ecosystem.',
]

print(f'Query: "{query}"')
print(f'Passages: {len(passages)}')
print('\nScoring...')

scores = []
for i, passage in enumerate(passages):
    tokens = tok([query], [passage], max_length=1024, padding='max_length',
                 truncation=True, return_tensors='np')
    inp_ids = httpclient.InferInput('input_ids',
                                    tokens['input_ids'].shape, 'INT64')
    inp_mask = httpclient.InferInput('attention_mask',
                                     tokens['attention_mask'].shape, 'INT64')
    inp_ids.set_data_from_numpy(tokens['input_ids'].astype(np.int64))
    inp_mask.set_data_from_numpy(tokens['attention_mask'].astype(np.int64))
    result = client.infer(model_name=MODEL_NAME,
                          inputs=[inp_ids, inp_mask])
    score = result.as_numpy('score')[0][0]
    scores.append((score, i, passage))

# Sort by score (highest = most relevant)
scores.sort(key=lambda x: x[0], reverse=True)

print(f'\n{"=" * 80}')
print(f'Reranking Results for: "{query}"')
print(f'{"=" * 80}')
for rank, (score, idx, passage) in enumerate(scores, 1):
    sigmoid_score = 1 / (1 + np.exp(-score))
    short = passage[:80] + '...' if len(passage) > 80 else passage
    print(f'\n  Rank {rank} (score={score:+.4f}, prob={sigmoid_score:.4f}):')
    print(f'    [{idx}] {short}')

print(f'\n{"=" * 80}')
print('Top-3 passages are about renewable energy, as expected.')

Query: "What are the benefits of renewable energy?"
Passages: 6

Scoring...

Reranking Results for: "What are the benefits of renewable energy?"

  Rank 1 (score=+1.9195, prob=0.8721):
    [0] Renewable energy sources like solar and wind power produce electricity without g...

  Rank 2 (score=+1.5806, prob=0.8293):
    [4] Wind energy creates more jobs per megawatt than coal or natural gas power plants...

  Rank 3 (score=+0.7861, prob=0.6870):
    [2] Solar panels have become 90% cheaper over the past decade, making renewable ener...

  Rank 4 (score=-3.3127, prob=0.0351):
    [1] The stock market experienced significant volatility in Q4 2025, with tech stocks...

  Rank 5 (score=-3.3220, prob=0.0348):
    [3] The history of ancient Rome spans over a thousand years, from its founding in 75...

  Rank 6 (score=-3.3224, prob=0.0348):
    [5] Python is a popular programming language known for its readability and extensive...

Top-3 passages are about renewable energy, as expected.


---
## Step 7: Cleanup

Stop and remove the Docker container.

In [8]:
subprocess.run(['docker', 'rm', '-f', 'triton-bert-reranker'], capture_output=True)
print('Triton server stopped and removed.')

Triton server stopped and removed.
